# PRUEBA DS GASES-HT

- Importamos las librerías necesarias:

In [222]:
import os
import pandas as pd
import torch
import torchaudio
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import librosa
import librosa.display
import IPython.display as ipd
import sklearn
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
import altair as alt
import plotly.graph_objs as go
import re
import calendar

## Análisis exploratorio de los datos (EDA)

Realizamos la importación de los datos:

In [223]:
DATOS=pd.read_csv(r"C:\Users\ANGIE VANESA\PROYECTOS\Prueba_Gases\data.csv")
DATOS.head()

,PERIODO,ID CLIENTE,DEPARTAMENTO,CLASIFICACION,CONSUMO,TIPO CONSUMO
0,1/1/2021,1068535,ATLANTICO,RESIDENCIAL,0,REAL
1,1/1/2021,1095302,ATLANTICO,COMERCIAL,21,REAL
2,1/1/2021,1131454,ATLANTICO,RESIDENCIAL,0,REAL
3,1/1/2021,1134487,ATLANTICO,COMERCIAL,132,REAL
4,1/1/2021,1140821,ATLANTICO,RESIDENCIAL,0,REAL


### Verificación de datos faltantes:

Vemos que los datos no tienen registros faltantes:

In [243]:
datos_faltantes=DATOS.isnull().sum()
datos_faltantes

PERIODO          0
ID CLIENTE       0
DEPARTAMENTO     0
CLASIFICACION    0
CONSUMO          0
TIPO CONSUMO     0
MES              0
dtype: int64

In [224]:
DATOS.shape

(312, 6)

La data está conformada por 312 registros y 6 columnas, cuyos nombres son:

In [225]:
DATOS.columns

Index(['PERIODO', 'ID CLIENTE', 'DEPARTAMENTO', 'CLASIFICACION', 'CONSUMO',
       'TIPO CONSUMO'],
      dtype='object')

### Variables

En los datos se tienen 6 variables: 

- Periodo: las fechas de registro de consumo mensual. 

- ID cliente: Código de identificación del cliente.

- Departamento: Territorio nacional donde se encuentra ubicado el cliente.

- Clasificación: Etiqueta asignada para categorizar al cliente según el tipo de cliente o el uso que se le da al consumo registrado.

- Consumo: Registro del consumo mensual de cada cliente.

- Tipo de consumo: Esta variable nos indica si el registro de consumo de los clientes fue real o estimado 


#### Análisis descriptivo univariado

Los datos corresponde al registro del consumo mensual de una lista de clientes, identificados con un código o ID diferente. Se realizó un filtro para identificar cuántos clientes se encuentran en la muestra obteniéndose:

In [226]:
DATOS['ID CLIENTE'].unique()

array([ 1068535,  1095302,  1131454,  1134487,  1140821,  1999658,
        2058257,  2076530,  2084470,  6089526,  6111795,  8091803,
        8092746, 17067567, 17186468, 50002330, 50043502, 50071540,
       50095570, 50157457, 50599279, 50689487, 51030818, 51390294,
       51577033, 51639236], dtype=int64)

In [227]:
len(DATOS['ID CLIENTE'].unique())

26

En total son 26 clientes, a los cuales se les hizo un registro mensual de su consumo. A continuación se realizará un análisis descriptivo de los clientes:

In [228]:
datos=DATOS[['ID CLIENTE', 'DEPARTAMENTO','CLASIFICACION']].drop_duplicates()
datos.head()

,ID CLIENTE,DEPARTAMENTO,CLASIFICACION
0,1068535,ATLANTICO,RESIDENCIAL
1,1095302,ATLANTICO,COMERCIAL
2,1131454,ATLANTICO,RESIDENCIAL
3,1134487,ATLANTICO,COMERCIAL
4,1140821,ATLANTICO,RESIDENCIAL


#### Departamento

In [229]:
pd.DataFrame(datos['DEPARTAMENTO'].describe(include=object)).T

,count,unique,top,freq
DEPARTAMENTO,26,2,ATLANTICO,19


Podemos ver que la variable ***Departamento*** es categórica  y tiene dos niveles, uno  es **Atlantico** el cual tiene mayor frecuencia, representando aproximadamente el 73% de los clientes registrados. A continuación un diagrama de barras: 

In [259]:
fig=px.bar(datos['DEPARTAMENTO'].value_counts().sort_index(),
           x=datos['DEPARTAMENTO'].value_counts().index,
           y=datos['DEPARTAMENTO'].value_counts().values,
           labels={'x':'Departamento', 'y':'Frecuencia'},
           title='Departamento: Frecuencia',
           color_discrete_sequence=['#17becf']).update_layout(xaxis_title='Departamento',
                                            yaxis_title='Frecuencia',
                                            xaxis=dict(categoryorder='total descending'),
                                            height=500, width=600)
fig.show()

Hay 7 registros de la categoría ***Magdalena*** en los datos, representando un 26,9%. Vemos que de los 26 clientes, gran parte son del departamento del Atlántico.

#### Clasificación

In [231]:
pd.DataFrame(datos['CLASIFICACION'].describe(include=object)).T

,count,unique,top,freq
CLASIFICACION,26,3,RESIDENCIAL,15


La variable ***Clasificacion*** tiene 3 niveles o categorías, siendo la categoría **Residencial** la más frecuente; se tiene en total 15 clientes con esta categoría, representando un 57,6% del total.

In [232]:
fig=px.bar(datos['CLASIFICACION'].value_counts().sort_index(),
           x=datos['CLASIFICACION'].value_counts().index,
           y=datos['CLASIFICACION'].value_counts().values,
           labels={'x':'Clasificación', 'y':'Frecuencia'},
           title='Clasificación: Frecuencia',
           color_discrete_sequence=['#17becf']).update_layout(xaxis_title='Clasificación',
                                            yaxis_title='Frecuencia',
                                            xaxis=dict(categoryorder='total descending'),
                                            height=500, width=600).show()

El diagrama de barras anterior nos muestra la frecuencia de cada categoría en la variable ***Clasificacion***. La segunda categoría es **Comercial** con 6 registros, mientras que la categoría **Industrial** cuenta con 5 registros. Vemos que los clientes se encuentran categorizados, y la mayoría de estos son parte del sector residencial.

#### Tipo de consumo

In [233]:
DATOS['TIPO CONSUMO'].unique()

array(['REAL', 'ESTIMADO'], dtype=object)

La variable ***Tipo consumo*** también es categórica, con dos categorías: 

- REAL

- ESTIMADO

- Tipo de consumo en el año:

Los datos entregan información sobre el tipo de consumo mensual en cada cliente durante el año 2021:

In [234]:
fig=px.bar(DATOS['TIPO CONSUMO'].value_counts().sort_index(),
           x=DATOS['TIPO CONSUMO'].value_counts().index,
           y=DATOS['TIPO CONSUMO'].value_counts().values,
           labels={'x':'Tipo de consumo', 'y':'Frecuencia'},
           title='Tipo de consumo: Frecuencia',
           color_discrete_sequence=['#17becf']).update_layout(xaxis_title='Tipo de consumo',
                                            yaxis_title='Frecuencia',
                                            xaxis=dict(categoryorder='total descending'),
                                            height=500, width=600).show()

El diagrama de barras anterior nos muestra que la categoría **Real** es la que más registros tuvo en la muestra. **Estimado** es la segunda categoría de esta variable, con una pequeña proporción de registros, menos del 3%. Gran parte de los clientes en la muestra contaron con un registro de consumo "real" durante el año 2021
.

- Tipo de consumo por periodo

In [236]:
DATOS['PERIODO'] = pd.to_datetime(DATOS['PERIODO'])
DATOS['MES'] = DATOS['PERIODO'].dt.strftime('%B')
meses_ord= ["January", "February", "March", "April", "May", "June", 
            "July", "August", "September", "October", "November", "December"]
Tipo_consumo=DATOS[['MES','TIPO CONSUMO']]
Tipo_consumo['MES'] = pd.Categorical(Tipo_consumo['MES'], categories=meses_ord, ordered=True)
Tipo_consumo=Tipo_consumo.groupby(['MES','TIPO CONSUMO']).size().reset_index(name='count')
Tipo_consumo.head()

,MES,TIPO CONSUMO,count
0,January,ESTIMADO,0
1,January,REAL,26
2,February,ESTIMADO,0
3,February,REAL,26
4,March,ESTIMADO,0


In [241]:
fig=px.bar(Tipo_consumo, x='MES', y='count', color='TIPO CONSUMO',
           labels={'count':'Frecuencia','TIPO CONSUMO':'Tipo de consumo','MES':'Meses'},
           title='Tipo de consumo por mes: Frecuencia',
           color_discrete_sequence=px.colors.qualitative.Set2).update_layout(yaxis=dict(dtick=5),barmode='group', 
                                                              height=500,
                                                              width=600).show()

El diagrama de barras anterior nos muestra que los primeros tres meses del año , mayo y diciembre, los 26 clientes registraron un tipo de consumo **real**, mientras que hubo meses donde algunos clientes tuvieron un tipo de consumo **estimado** (*En verde*)

#### CONSUMO

- Consumo anual en la muestra:

El siguiente resumen corresponde a un análisis descriptivo de la variable numérica consumo:

In [252]:
pd.DataFrame(DATOS['CONSUMO'].describe()).T

,count,mean,std,min,25%,50%,75%,max
CONSUMO,312.0,1226.971154,3234.101606,-75.0,11.0,20.0,353.75,23748.0


- El registro de consumo promedio en el año 2021 fue de 1226,97 con una desviación estándar de 3234.10; esta desviación nos indica una gran variabilidad en los datos. 

- El 50% del consumo anual fue menor o igual a 20. Dado que la mediana de los datos es muy inferior a la media, hay un sesgo positivo, es decir, 
